In [ ]:
!cp drive/MyDrive/Colab\ Notebooks/Plots.py .

In [ ]:
import os
import glob
import re
import pandas as pd
import numpy as np
from Plots import MultiRunCorrelationPlotter

In [ ]:
base_path = "/content/drive/MyDrive/Thesis"
ce_folders = glob.glob(os.path.join(base_path, "CE*"))

df_dict = {}

for folder in ce_folders:
    csv_files = glob.glob(os.path.join(folder, "RCS_*_5S2P_*_Run*_*.csv"))

    for file_path in csv_files:
        match = re.search(r"RCS_\d+_\d+_5S2P_(.*?)_(Run\d)_\d\.csv", os.path.basename(file_path))
        if match:
            tag = match.group(1)
            run = match.group(2)
            key = f"df_{tag}_{run}"

            df = pd.read_csv(file_path)

            df_dict[key] = df
            print(f"Loaded {key} from {file_path}")
        else:
            print(f"Skipped unmatched file: {file_path}")

Loaded df_50CE_Run2 from /content/drive/MyDrive/Thesis/CE Part2/RCS_250414_1317_5S2P_50CE_Run2_1.csv
Loaded df_10CE_Run2 from /content/drive/MyDrive/Thesis/CE Part2/RCS_250414_1442_5S2P_10CE_Run2_1.csv
Loaded df_05CE_Run2 from /content/drive/MyDrive/Thesis/CE Part2/RCS_250421_1312_5S2P_05CE_Run2_1.csv
Loaded df_05CE_Run3 from /content/drive/MyDrive/Thesis/CE Part3/RCS_250416_1643_5S2P_05CE_Run3_1.csv
Loaded df_05CE_Run4 from /content/drive/MyDrive/Thesis/CE Part3/RCS_250416_1355_5S2P_05CE_Run4_1.csv
Loaded df_05CE_Run5 from /content/drive/MyDrive/Thesis/CE Part3/RCS_250416_1229_5S2P_05CE_Run5_1.csv
Loaded df_05CE_Run6 from /content/drive/MyDrive/Thesis/CE Part3/RCS_250415_1506_5S2P_05CE_Run6_1.csv
Loaded df_50CE_Run1 from /content/drive/MyDrive/Thesis/CE Part1/RCS_250410_1612_5S2P_50CE_Run1_1.csv
Loaded df_10CE_Run1 from /content/drive/MyDrive/Thesis/CE Part1/RCS_250414_1016_5S2P_10CE_Run1_1.csv
Loaded df_05CE_Run1 from /content/drive/MyDrive/Thesis/CE Part1/RCS_250414_1115_5S2P_05CE_R

In [ ]:
for key, df in df_dict.items():
    dataset_name_full = key.replace("df_", "").replace("_", " ").strip()
    # dataset_name_folder = dataset_name_full.lower().split()[0]
    analyzer = CorrelationAnalyzer(
        df,
        dataset_name=dataset_name_full,
        dataset_folder='CE',
        desired_window_in_seconds=60,
        step_fraction=0.8
    )
    print(f"****** Running analysis for {dataset_name_full} ******")
    analyzer.analyze_all()


****** Running analysis for 50CE Run2 ******
****** Running analysis for 10CE Run2 ******
****** Running analysis for 05CE Run2 ******
****** Running analysis for 05CE Run3 ******
****** Running analysis for 05CE Run4 ******
****** Running analysis for 05CE Run5 ******
****** Running analysis for 05CE Run6 ******
****** Running analysis for 50CE Run1 ******
****** Running analysis for 10CE Run1 ******
****** Running analysis for 05CE Run1 ******


In [ ]:
import re
from collections import defaultdict

def group_dataframes_by_fault(df_dict):
    fault_groups = defaultdict(dict)
    for name, df in df_dict.items():
        # Use re.IGNORECASE and correct pattern to match both 'Run' and 'run'
        match = re.match(r"df_([a-zA-Z0-9]+)_(run\d+)", name, flags=re.IGNORECASE)
        if match:
            fault_type = match.group(1).lower()  # Convert to lowercase for consistency
            run_id = match.group(2).lower()
            fault_groups[fault_type][run_id] = df
        else:
            print(f"Skipped unmatched key: {name}")
    return dict(fault_groups)


In [ ]:
# Step 1: Automatically group by fault type
faultwise_df_dict = group_dataframes_by_fault(df_dict)
print(faultwise_df_dict.keys())

# Step 2: Run correlation analysis on each fault type
analyzer = CorrelationAnalyzer(df_dict, output_path="/content/drive/MyDrive/Thesis/Plots")

for fault_type, fault_dataframes in faultwise_df_dict.items():
    print(f"****** Running analysis for {fault_type} ******")
    analyzer.df_dict = fault_dataframes  # Set only relevant fault's dataframes
    analyzer.analyze_fault(fault_type)

<re.Match object; span=(0, 12), match='df_50CE_Run2'>
<re.Match object; span=(0, 12), match='df_10CE_Run2'>
<re.Match object; span=(0, 12), match='df_05CE_Run2'>
<re.Match object; span=(0, 12), match='df_05CE_Run3'>
<re.Match object; span=(0, 12), match='df_05CE_Run4'>
<re.Match object; span=(0, 12), match='df_05CE_Run5'>
<re.Match object; span=(0, 12), match='df_05CE_Run6'>
<re.Match object; span=(0, 12), match='df_50CE_Run1'>
<re.Match object; span=(0, 12), match='df_10CE_Run1'>
<re.Match object; span=(0, 12), match='df_05CE_Run1'>
Detected fault types: ['50ce', '10ce', '05ce']
dict_keys(['50ce', '10ce', '05ce'])
****** Running analysis for 50ce ******
No runs found for fault type '50ce'
****** Running analysis for 10ce ******
No runs found for fault type '10ce'
****** Running analysis for 05ce ******
No runs found for fault type '05ce'


In [ ]:
plotter = MultiRunCorrelationPlotter(df_dict)
plotter.analyze_all()

Grouping dataframes by fault type...
****** Running analysis for 50CE ******
****** Running analysis for 10CE ******
****** Running analysis for 05CE ******
